In [ ]:
%%capture
# ⬇️ Install dependencies (Colab-compatible)
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
from torch import __version__ as torch_version
from packaging.version import Version as V
xformers_version = "xformers==0.0.27" if V(torch_version) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers_version} trl transformers peft accelerate bitsandbytes triton gradio scikit-learn datasets
!pip install -U bitsandbytes

# ✅ Load and Clean Dataset
import re, json, requests
from datasets import Dataset

url = "https://raw.githubusercontent.com/ParthDesai1719/kokoui_comp_gen/main/koko_ui.jsonl"
raw = requests.get(url).text.strip().split("\n")

cleaned = []
for line in raw:
    item = json.loads(line)
    code = re.sub(r"//.*?$", "", item["code"], flags=re.MULTILINE)
    code = re.sub(r"/\*.*?\*/", "", code, flags=re.DOTALL)
    cleaned.append({
        "input": item["prompt"].strip(),
        "output": code.strip(),
        "text": f"<|user|>\n{item['prompt'].strip()}\n<|assistant|>\n{code.strip()}"
    })

hf_dataset = Dataset.from_list(cleaned)

# ✅ Load Model
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct",
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)

tokenizer.pad_token = tokenizer.eos_token

# ✅ Tokenize
from transformers import TrainingArguments, Trainer

def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors=None
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = hf_dataset.map(tokenize_function, batched=True, remove_columns=hf_dataset.column_names)

# ✅ Apply LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj", "lm_head",
    ],
)

# ✅ Train
training_args = TrainingArguments(
    output_dir="shadcn_llama3_finetuned_trainer",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    max_steps=250,
    learning_rate=2e-5,
    fp16=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="no",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()
FastLanguageModel.for_inference(model)

    PyTorch 2.7.0+cu126 with CUDA 1206 (you have 2.6.0+cu124)
    Python  3.11.12 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [ ]:
import gradio as gr
import torch
def generate_shadcn(prompt: str) -> str:
    # Construct prompt with consistent structure
    full_prompt = f"<|user|>\n{prompt.strip()}\n<|assistant|>\n"

    # Tokenize prompt
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,  # disable randomness
            top_p=0.95,
            temperature=0.7,
            repetition_penalty=1.2,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )

    # Decode output
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Strip to only get assistant's response
    if "<|assistant|>" in decoded:
        result = decoded.split("<|assistant|>")[-1].strip()
    else:
        result = decoded

    # Final fallback
    return result if result and len(result) > 10 else "⚠️ Output too short or malformed. Try again or fine-tune better."


gr.Interface(
    fn=generate_shadcn,
    inputs=gr.Textbox(
        lines=3,
        placeholder="Describe the ShadCN component you want (e.g. Create a dialog with title, description, and confirm button)",
        label="Your Prompt"
    ),
    outputs=gr.Code(label="Generated ShadCN Component", language="javascript"),
    title="🛠️ ShadCN Component Generator",
    description="Enter a descriptive prompt to generate a React component using ShadCN UI. No explanation, just clean code."
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://030d10804c77c7788d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
